# fILterless245Days

In [1]:

import dask.dataframe as dd
# loading metadata file
filePath = '/'
df = dd.read_csv(filePath + '.csv').drop_duplicates()

# Pre-processing to filter out inconsistent data
df = df[df['Number of Occupants'] < df['Number of Occupants'].quantile(.999)]
df = df[df['Number of Floors'] < df['Number of Floors'].quantile(.999)]
df = df[df['Floor Area [ft2]'] < df['Floor Area [ft2]'].quantile(.999)]
df = df[df['Number of Remote Sensors'] < df['Number of Remote Sensors'].quantile(.999)]
df = df[(df['Country']=='US') | (df['Country']=='CA')]

In [2]:
from pathlib import Path
lista = []
# List all files in directory using pathlib
basepath = Path("1/")
files_in_basepath = (entry for entry in basepath.iterdir() if entry.is_file())
for item in files_in_basepath:
    lista.append(item.name)
basepath = Path("2/")
files_in_basepath = (entry for entry in basepath.iterdir() if entry.is_file())
for item in files_in_basepath:
    lista.append(item.name)
basepath = Path("3/")
files_in_basepath = (entry for entry in basepath.iterdir() if entry.is_file())
for item in files_in_basepath:
    lista.append(item.name)
basepath = Path("4/")
files_in_basepath = (entry for entry in basepath.iterdir() if entry.is_file())
for item in files_in_basepath:
    lista.append(item.name)

In [4]:
lista_FileName = [s.replace('.csv', '') for s in lista]  # to have only the name of the file

In [5]:
# Create dataset with only Identifiers in 2019 
df_in_2019 = df[df.Identifier.isin(lista_FileName)]

In [6]:
# Creation of datasets for 1 and 2 occupants
df1 = df_in_2019[(df_in_2019['Number of Occupants'] == 1 ) & (df_in_2019['Number of Remote Sensors'] >= 3)]
df2 = df_in_2019[(df_in_2019['Number of Occupants'] == 2 ) & (df_in_2019['Number of Remote Sensors'] >= 3)]

In [7]:
df2 = df2.sample(frac=(len(df1)/len(df2)), random_state=0)

In [11]:
# Take all Identifiers of df2, to after open them
a=df2.loc[:,'Identifier'].compute().tolist()
lista=[]
for i in a:
    lista.append(mainPath+i+'.csv')

In [12]:
# Read all files in the df2['Identifier'] column
ff=dd.read_csv(lista, usecols = use_cols, parse_dates=['DateTime'], include_path_column=True)
ff['path']=ff['path'].str.replace("/", '')
ff['path']=ff['path'].str.replace('.csv', '')

In [13]:
# Manage of columns

df_motion = ff.loc[:, ff.columns.str.contains('_Motion')]
ff['working_sensors'] = 11-df_motion.isna().sum(axis=1)
ff['activation'] = ff.loc[:, ff.columns.str.contains('_Motion')].sum(axis=1) # sumo cantidad de rs en 1
ff['dayofweek']=ff['DateTime'].dt.day_name()
ff['Time'] = ff['DateTime'].dt.time
ff['Month'] = ff['DateTime'].dt.month



In [14]:
ff_columns = ['path', 'activation', 'dayofweek', 'Time', 'Month','working_sensors']
result = dd.merge(df2[['Identifier','Number of Remote Sensors']], ff[ff_columns], left_on='Identifier', right_on='path').drop(['path'],axis=1)
result['mean_activation']=result['activation']/(result['working_sensors']) 

In [15]:
aa=result[['Identifier','dayofweek','Time','mean_activation']].groupby(['Identifier','Time','dayofweek']).mean()
pp=dd.from_array(aa.to_records()) 

In [16]:
pp["TimeSTR"]=pp["Time"].apply((lambda x: x.strftime('%H:%M')), meta='str')
aa2 = pp.copy()
new = aa2["TimeSTR"]
  
# concatenating team with name column 
# overwriting name column 
aa2["dayofweek"]= aa2["dayofweek"].str.cat(new, sep =", ") 

In [17]:

bb=aa2.drop(['Time'],axis=1)
bb = bb.categorize(columns='dayofweek')
table = dd.pivot_table(bb, values='mean_activation', index='Identifier',
                    columns='dayofweek')

In [18]:
t=dd.from_array(table.to_records())
result_f = dd.merge(df2,t, on='Identifier').drop(['filename'],axis=1)

In [19]:
(result_f.compute()).to_csv('1.csv')
